In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Steps:** 
    
    1)  Import libraries.

    2)  Load data (and split into training & validation sets).

    2.5)  Plot some images to see what the data looks like.

    3) Set up a pre-train base of choice: MobileNetV2.

    4) Attach extra layers to the base.

    5) Train the model.

    6) Create confusion matrix & classification performance.

    7) Inspect correct and incorrect classification.


**To improve on later:** 

    1) Create a test set & evaluate performance on test set (Step 2, 6, 7)
    2) Find a way to extract performance from training better.
    3) Compare with different pre-trained base, other model configurations.

### 1) Import libraries

In [ ]:
# import basic libraries 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path

# import tensorflow  
from tensorflow import keras
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow import data

from sklearn.metrics import confusion_matrix, classification_report

# from keras.applications.resnet50 import ResNet50
# from keras.applications.resnet50 import preprocess_input, decode_predictions
# from tensorflow.keras.layers.experimental import preprocessing

# Plotting 
import matplotlib.pyplot as plt

### 2) Load data

In [ ]:
# set path to data directory
path = Path('/kaggle/input/rice-image-dataset/Rice_Image_Dataset')

# check files in the directory
list(path.glob('*'))

In [ ]:
# Set parameters
IMAGE_SIZE=(224, 224)
BATCH_SIZE=32


# Create training and validation sets
orig_data = keras.preprocessing.image_dataset_from_directory(directory=path,
                                                        labels='inferred',
                                                        label_mode ='categorical',
                                                        image_size=IMAGE_SIZE,
                                                        subset='both',
                                                        batch_size=BATCH_SIZE,
                                                        validation_split=0.2,
                                                        shuffle=True,
                                                        seed=42)

train_dataset = orig_data[0]
validation_dataset = orig_data[1]

In [ ]:
# Check number of batches
print('Number of training batches: %d' % data.experimental.cardinality(train_dataset))
print('Number of validation  batches: %d' % data.experimental.cardinality(validation_dataset))

In [ ]:
# Check class distribution


### 2.5) Plot some images

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(15, 4))
for images, labels in train_dataset.take(1): # take data from batch 1 (32 items)
    for i in range(20):
        ax = plt.subplot(2, 10, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        ind = np.where(labels[i] == 1)[0][0]
        plt.title(class_names[ind])
#         plt.title(class_names[labels[i]])
        plt.axis("off")

### 3) Set up a pre-trained base

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMAGE_SHAPE = IMAGE_SIZE + (3,)
base_model = keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
# Check features extracted from the base
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# set trainable to False
base_model.trainable = False

# Check the structure of the base model
# base_model.summary()


In [ ]:
# # Add global average pooling layer 
global_average_layer = keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(base_model.output)

print(feature_batch_average.shape)


# Add prediction later
prediction_layer = keras.layers.Dense(5, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

### 4) Attach extra layers to the base

In [ ]:
# Putting all the layers together
model = keras.Sequential([base_model,
                          global_average_layer,
                          prediction_layer
])

In [ ]:
model.summary(line_length=100)


### 5) Train the model

In [ ]:
optimizer = keras.optimizers.Adam(epsilon=0.0001)
model.compile(
    optimizer=optimizer,
    loss = "categorical_crossentropy",
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

In [ ]:
# Evaluate on validation dataset (should ideally be independent test set)
# loss0, accuracy0 = model.evaluate(validation_dataset)
# print("initial loss: {:.2f}".format(loss0))
# print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
# Check on possible keys
print(history.history.keys())

In [ ]:
# Plot learning curves for training and validation data

# Accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Loss
loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot training & validation accuracy
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

# Plot training & validation loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Categorical Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### 6) Confusion matrix & classification performance

In [ ]:
# Retrieve a batch of images from the validation set
image_batch, label_batch = validation_dataset.as_numpy_iterator().next()
predictions = model.predict(image_batch)

# Formatting
y_true = np.argmax(label_batch, axis=1)
y_pred = np.argmax(predictions, axis=1)

In [ ]:
#Confution Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))
print('----------------')
print('Classification Report')
target_names = validation_dataset.class_names
print(classification_report(y_true, y_pred, target_names=target_names))

### 7) Examples of correct and incorrect classification

In [ ]:
plt.figure(figsize=(15, 10))
for images, labels in validation_dataset.take(1):
    for i in range(20):
        ax = plt.subplot(5, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        ind = np.where(labels[i] == 1)[0][0]
        y_pred_ind = y_pred[i]
        plt.title(class_names[ind] + " (T), " + class_names[y_pred_ind] + " (P) ")
        plt.axis("off")

Codes are adapted from various sources: 
    
    - https://www.tensorflow.org/tutorials/images/transfer_learning